# Feature Selection

변수 선택

Feature Selection은 주어진 고차원 데이터 분석 시에 모델링에 사용할 변수를 선택하거나 제거하는 방법입니다. 

## 변수 자체의 Variance Threshold 를 통한 변수 제거

Feature Selection의 가장 기본적인 접근방법으로, Variance 가 Threshold 보다 작은 변수를 제거합니다. Variance 가 낮다는 것은 변수 안에 같은 값의 데이터가 많다는 의미입니다. 예를 들면 변수의 데이터의 다수가 0 으로 채워져 있는 경우입니다.

Variance Threshold 는 Independent Variable(독립변수, x)와 Dependent Variable(종속변수, y)간의 관계를 고려하지 않기 때문에 분석 차원에서 제거시 이 부분을 고려해봐야 합니다.


In [1]:
import numpy as np
from sklearn.feature_selection import VarianceThreshold

np_x = np.array(
    [[0, 2, 0, 3],
     [0, 1, 4, 3],
     [0, 1, 1, 3]]
)
print(np_x)

selector = VarianceThreshold()
fit = selector.fit(np_x)
print(fit.variances_)

np_x_selected = selector.fit_transform(np_x)
print(np_x_selected.shape)
print(np_x_selected)

[[0 2 0 3]
 [0 1 4 3]
 [0 1 1 3]]
[0.         0.22222222 2.88888889 0.        ]
(3, 2)
[[2 0]
 [1 4]
 [1 1]]


## Independet Variable와 Dependent Variable간의 관계를 통해 변수 제거

주어진 고차원 Independent Variable 각각을 Dependent Variable와의 관계를 계산하여 그 중 모델의 성능에 영향을 줄수 있는 독립변수를 우선적으로 선택합니다. 독립변수와 종속변수간의 관계가 강한 Top-K 변수를 선택함으로써 아래와 같은 이점을 얻을 수 있습니다.

* Overfitting 감소
* 모델 성능 향상
* 모델 훈련 시간 감소

Scikit-learn 에서는 단변량 통계 (Univariate Statistics)를 이용한 선택 방법 및 종속 변수의 범주에 따른 분석 방법이 Regression, Classification 이냐에 따라 다른 특징 선택 방법을 제공합니다.

변수 선택 방법
* SelectKBest: 가장 높은 점수를 받은 K 개 변수외의 변수를 제거
* SelectPercentile: 가장 높은 점수순으로 사용자가 제공한 Percent 를 제외한 변수를 제거
* SelectFpr: False Positve Rate를 통해 변수를 제거
* SelectFdr: False Discovery Rate를 통해 변수를 제거
* SelectFwe: Family Wise Error를 통해 변수를 제거

종속 변수 분석 방법
* Regression: f_regression, mutual_info_regression
* Classification: chi2, f_classif, mutual_info_classif

Regression 문제에 Classification 특징 선택 함수를 쓸 경우에는 잘못된 결과를 얻을 수 있기 때문에 주의해야 합니다. 한가지 더 생각해보아야 할 점은 종속변수와 관계가 약한 독립변수의 제거에 있습니다. 많은 특징 데이터를 학습하여 복잡한 모델을 만들게 되는 딥러닝 기법에서는 약한 관계에 있는 변수들도 사용 시에 도움이 될 수 있기에 Top-K 변수를 사용하는 것 보다는 전혀 관계가 없는 변수만을 제거하는 것을 분석 차원 축소의 의미로 사용하는 것을 생각해보아야 합니다.

### Regression

Scikit-learn 에서 제공하는 샘플 데이터 셋에서 Regression 에 적합한 데이터 셋에 대해서 특징 선택을 수행해봅니다. 데이터 셋 로딩 함수의 도움말을 통해 데이터 컬럼 정보를 확인해 봅니다.

In [2]:
from sklearn.datasets import load_boston

load_boston?

Signature: load_boston(return_X_y=False)
Docstring:
Load and return the boston house-prices dataset (regression).

==============     ==============
Samples total                 506
Dimensionality                 13
Features           real, positive
Targets             real 5. - 50.
==============     ==============

Read more in the :ref:`User Guide <boston_dataset>`.

Parameters
----------
return_X_y : boolean, default=False.
    If True, returns ``(data, target)`` instead of a Bunch object.
    See below for more information about the `data` and `target` object.

    .. versionadded:: 0.18

Returns
-------
data : Bunch
    Dictionary-like object, the interesting attributes are:
    'data', the data to learn, 'target', the regression targets,
    'DESCR', the full description of the dataset,
    and 'filename', the physical location of boston
    csv dataset (added in version `0.20`).

(data, target) : tuple if ``return_X_y`` is True

    .. versionadded:: 0.18

Notes
-----
    .. v

전체 13개의 변수 중에 5개의 특징을 선택해 보도록 하겠습니다.

In [3]:
import pandas as pd
pd.options.display.float_format = '{:.15f}'.format

import numpy as np

from sklearn.datasets import load_boston
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

np_x, np_y = load_boston(return_X_y=True)
print("np_x.shape={}, np_y.shape={}".format(np_x.shape, np_y.shape))

selector = SelectKBest(score_func = f_regression, k = 'all')
fit = selector.fit(np_x, np_y)

# Display score
df_columns = pd.DataFrame(np.arange(np_x.shape[1]))
df_scores = pd.DataFrame(fit.scores_)
df_feature_scores = pd.concat([df_columns, df_scores], axis = 1)
df_feature_scores.columns = ['column', 'score']
display(df_feature_scores)

# Select Top-N score
display(df_feature_scores.nlargest(5, 'score'))

index =  df_feature_scores.nlargest(5, 'score')['column'].values
display(index)

np_x_selected = np_x[:, index]
print(np_x_selected.shape)
display(np_x_selected[:5])

np_x.shape=(506, 13), np_y.shape=(506,)


,column,score
0,0,89.486114757681179
1,1,75.257642298954011
2,2,153.954883136110595
3,3,15.971512420371932
4,4,112.591480279706715
5,5,471.846739876478978
6,6,83.477459219237133
7,7,33.579570325905010
8,8,85.914277669840885
9,9,141.761356577424152


,column,score
12,12,601.617871109907583
5,5,471.846739876478978
10,10,175.105542875713638
2,2,153.954883136110595
9,9,141.761356577424152


array([12,  5, 10,  2,  9])

(506, 5)


array([[  4.98 ,   6.575,  15.3  ,   2.31 , 296.   ],
       [  9.14 ,   6.421,  17.8  ,   7.07 , 242.   ],
       [  4.03 ,   7.185,  17.8  ,   7.07 , 242.   ],
       [  2.94 ,   6.998,  18.7  ,   2.18 , 222.   ],
       [  5.33 ,   7.147,  18.7  ,   2.18 , 222.   ]])

### Classification

Scikit-learn 에서 제공하는 샘플 데이터 셋에서 Classication 에 적합한 데이터 셋에 대해서 특징 선택을 수행해봅니다. 데이터 셋 로딩 함수의 도움말을 통해 데이터 컬럼 정보를 확인해 봅니다.

In [4]:
from sklearn.datasets import load_wine

load_wine?

Signature: load_wine(return_X_y=False)
Docstring:
Load and return the wine dataset (classification).

.. versionadded:: 0.18

The wine dataset is a classic and very easy multi-class classification
dataset.

=================   ==============
Classes                          3
Samples per class        [59,71,48]
Samples total                  178
Dimensionality                  13
Features            real, positive
=================   ==============

Read more in the :ref:`User Guide <wine_dataset>`.

Parameters
----------
return_X_y : boolean, default=False.
    If True, returns ``(data, target)`` instead of a Bunch object.
    See below for more information about the `data` and `target` object.

Returns
-------
data : Bunch
    Dictionary-like object, the interesting attributes are: 'data', the
    data to learn, 'target', the classification labels, 'target_names', the
    meaning of the labels, 'feature_names', the meaning of the features,
    and 'DESCR', the full description of the

전체 13개의 변수 중에 5개의 특징을 선택해 보도록 하겠습니다.

In [5]:
import pandas as pd
pd.options.display.float_format = '{:.15f}'.format

import numpy as np

from sklearn.datasets import load_wine
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

np_x, np_y = load_wine(return_X_y=True)
print("np_x.shape={}, np_y.shape={}".format(np_x.shape, np_y.shape))

selector = SelectKBest(score_func = f_classif, k = 'all')
fs_fit = selector.fit(np_x, np_y)

# Display score
df_columns = pd.DataFrame(np.arange(np_x.shape[1]))
df_scores = pd.DataFrame(fit.scores_)
df_feature_scores = pd.concat([df_columns, df_scores], axis = 1)
df_feature_scores.columns = ['column', 'score']
display(df_feature_scores)

# Select Top-N score
display(df_feature_scores.nlargest(5, 'score'))

index =  df_feature_scores.nlargest(5, 'score')['column'].values
display(index)

np_x_selected = np_x[:, index]
print(np_x_selected.shape)
display(np_x_selected[:5])

np_x.shape=(178, 13), np_y.shape=(178,)


,column,score
0,0,89.486114757681179
1,1,75.257642298954011
2,2,153.954883136110595
3,3,15.971512420371932
4,4,112.591480279706715
5,5,471.846739876478978
6,6,83.477459219237133
7,7,33.579570325905010
8,8,85.914277669840885
9,9,141.761356577424152


,column,score
12,12,601.617871109907583
5,5,471.846739876478978
10,10,175.105542875713638
2,2,153.954883136110595
9,9,141.761356577424152


array([12,  5, 10,  2,  9])

(178, 5)


array([[1.065e+03, 2.800e+00, 1.040e+00, 2.430e+00, 5.640e+00],
       [1.050e+03, 2.650e+00, 1.050e+00, 2.140e+00, 4.380e+00],
       [1.185e+03, 2.800e+00, 1.030e+00, 2.670e+00, 5.680e+00],
       [1.480e+03, 3.850e+00, 8.600e-01, 2.500e+00, 7.800e+00],
       [7.350e+02, 2.800e+00, 1.040e+00, 2.870e+00, 4.320e+00]])

## Recursive 한 변수 조합 평가를 통한 변수 제거

이 방법은 전체 종속 변수에 대한 서브셋을 만들고 그중 가장 나쁜 성능을 나타내는 변수 조합을 제거하는 방법입니다. 예를 들어 5개의 변수 중 3개의 변수를 선택한다고 하면, 재귀적으로 표현 가능한 변수의 조합 셋을 만들고, n-1 조합에서 가장 좋은 셋을 찾고 그 셋에서 n-2 조합 중에 가장 좋은 셋을 찾는 식입니다. 평가에 사용하는 함수는 원하는 모델 함수를 파라미터로 선택하여 이용할 수 있습니다. 이 방법은 변수의 수가 많은 고차원 데이터 셋에서는 많은 시간이 들기 때문에 고차원 데이터라면 추천되지는 않습니다.

### Regression

In [6]:
import pandas as pd
pd.options.display.float_format = '{:.15f}'.format

import numpy as np

from sklearn.datasets import load_boston
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

np_x, np_y = load_boston(return_X_y=True)
print("np_x.shape={}, np_y.shape={}".format(np_x.shape, np_y.shape))

estimator = SVR(kernel="linear")
selector = RFE(estimator, 5)
%time fit = selector.fit(np_x, np_y)

# Display score
df_columns = pd.DataFrame(np.arange(np_x.shape[1]))
df_ranks = pd.DataFrame(fit.ranking_)
df_feature_ranks = pd.concat([df_columns, df_ranks], axis = 1)
df_feature_ranks.columns = ['column', 'rank']
display(df_feature_ranks)

# Select Top-N score
index = fit.support_
display(index)

np_x_selected = np_x[:, index]
print(np_x_selected.shape)
display(np_x_selected[:5])

np_x.shape=(506, 13), np_y.shape=(506,)
CPU times: user 5.9 s, sys: 0 ns, total: 5.9 s
Wall time: 5.89 s


,column,rank
0,0,3
1,1,5
2,2,4
3,3,1
4,4,1
5,5,1
6,6,6
7,7,2
8,8,8
9,9,9


array([False, False, False,  True,  True,  True, False, False, False,
       False,  True, False,  True])

(506, 5)


array([[ 0.   ,  0.538,  6.575, 15.3  ,  4.98 ],
       [ 0.   ,  0.469,  6.421, 17.8  ,  9.14 ],
       [ 0.   ,  0.469,  7.185, 17.8  ,  4.03 ],
       [ 0.   ,  0.458,  6.998, 18.7  ,  2.94 ],
       [ 0.   ,  0.458,  7.147, 18.7  ,  5.33 ]])

### Classification

In [7]:
import pandas as pd
pd.options.display.float_format = '{:.15f}'.format

import numpy as np

from sklearn.datasets import load_wine
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

np_x, np_y = load_wine(return_X_y=True)
print("np_x.shape={}, np_y.shape={}".format(np_x.shape, np_y.shape))

estimator = SVC(kernel="linear")
selector = RFE(estimator, 5)
%time fit = selector.fit(np_x, np_y)

# Display score
df_columns = pd.DataFrame(np.arange(np_x.shape[1]))
df_ranks = pd.DataFrame(fit.ranking_)
df_feature_ranks = pd.concat([df_columns, df_ranks], axis = 1)
df_feature_ranks.columns = ['column', 'rank']
display(df_feature_ranks)

# Select Top-N score
index = fit.support_
display(index)

np_x_selected = np_x[:, index]
print(np_x_selected.shape)
display(np_x_selected[:5])

np_x.shape=(178, 13), np_y.shape=(178,)
CPU times: user 71.9 ms, sys: 0 ns, total: 71.9 ms
Wall time: 71.6 ms


,column,rank
0,0,1
1,1,4
2,2,1
3,3,5
4,4,8
5,5,3
6,6,1
7,7,1
8,8,6
9,9,2


array([ True, False,  True, False, False, False,  True,  True, False,
       False, False,  True, False])

(178, 5)


array([[14.23,  2.43,  3.06,  0.28,  3.92],
       [13.2 ,  2.14,  2.76,  0.26,  3.4 ],
       [13.16,  2.67,  3.24,  0.3 ,  3.17],
       [14.37,  2.5 ,  3.49,  0.24,  3.45],
       [13.24,  2.87,  2.69,  0.39,  2.93]])

## Modeling 을 통한 Feature selection

이 방법은 모델 알고리즘 중에 훈련시에 변수의 중요도가 뽑혀저 나오는 알고리즘을 이용하여 중요도에 따라서 Feature를 선택하는 방법입니다. 이 방법의 장점은 Feature 선택과 동시에 Baseline 모델로도 활용할 수 있다는 것입니다.

### Regression

estimator로 LinearRegression을 사용해도 되지만 좀 더 좋은 선택을 위해 Lasso Regression를 사용해봅니다. Lasso는 L1 Regularization 을 통해 모델의 복잡성을 줄이면서 Overfit 을 방지하는 Regression 알고리즘입니다. 이 알고리즘을 통해 모델을 훈련시킬때 사용되는 coefficient 를 통해 변수를 선택합니다. 사용할 수 있는 다른 estimator는 LogisticRegression(linear_model.LogisticRegression), LinearSVM(svm.LinearSVC) 이 있습니다.

In [8]:
import pandas as pd
pd.options.display.float_format = '{:.15f}'.format

import numpy as np

from sklearn.datasets import load_boston
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

np_x, np_y = load_boston(return_X_y=True)
print("np_x.shape={}, np_y.shape={}".format(np_x.shape, np_y.shape))

estimator = LassoCV(cv=5) # cv: cross validation
selector = SelectFromModel(estimator, threshold=0.25)
%time fit = selector.fit(np_x, np_y)

np_x_seleted = fit.transform(np_x)
print("np_x_seleted.shape={}".format(np_x_seleted.shape))

np_x.shape=(506, 13), np_y.shape=(506,)
CPU times: user 34.6 ms, sys: 213 µs, total: 34.8 ms
Wall time: 34.2 ms
np_x_seleted.shape=(506, 5)


### Classification

estimator로 Tree 기반 알고리즘인 RandomForest를 사용해봅니다.

In [9]:
import pandas as pd
pd.options.display.float_format = '{:.15f}'.format

import numpy as np

from sklearn.datasets import load_wine
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

np_x, np_y = load_wine(return_X_y=True)
print("np_x.shape={}, np_y.shape={}".format(np_x.shape, np_y.shape))

estimator = RandomForestClassifier(n_estimators=100)
selector = SelectFromModel(estimator, threshold='1.25*median')
%time fit = selector.fit(np_x, np_y)

np_x_seleted = fit.transform(np_x)
print("np_x_seleted.shape={}".format(np_x_seleted.shape))

np_x.shape=(178, 13), np_y.shape=(178,)
CPU times: user 60.2 ms, sys: 5 µs, total: 60.2 ms
Wall time: 59.7 ms
np_x_seleted.shape=(178, 5)
